In [3]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import pandas as pd
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [13]:
import numpy as np 
df['Glucose'] = np.where(df['Glucose']==0,df['Glucose'].median(),df['Glucose'])
df['Insulin'] = np.where(df['Insulin']==0,df['Insulin'].median(),df['Insulin'])
df['SkinThickness'] = np.where(df['SkinThickness']==0,df['SkinThickness'].median(),df['SkinThickness'])


In [14]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72,35.0,30.5,33.6,0.627,50,1
1,1,85.0,66,29.0,30.5,26.6,0.351,31,0
2,8,183.0,64,23.0,30.5,23.3,0.672,32,1
3,1,89.0,66,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40,35.0,168.0,43.1,2.288,33,1


In [15]:
y = df['Outcome']
X = df.drop('Outcome',axis=1)

In [16]:
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148.0,72,35.0,30.5,33.6,0.627,50
1,1,85.0,66,29.0,30.5,26.6,0.351,31
2,8,183.0,64,23.0,30.5,23.3,0.672,32
3,1,89.0,66,23.0,94.0,28.1,0.167,21
4,0,137.0,40,35.0,168.0,43.1,2.288,33


In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [21]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=10)
model.fit(X_train,y_train)
y_predict = model.predict(X_test)

In [29]:
from sklearn import metrics
cm = metrics.confusion_matrix(y_test,y_predict)
print(cm)
accuracy = metrics.accuracy_score(y_test,y_predict)
print(accuracy)
report = metrics.classification_report(y_test,y_predict)
print(report)

[[93 14]
 [20 27]]
0.7792207792207793
              precision    recall  f1-score   support

           0       0.82      0.87      0.85       107
           1       0.66      0.57      0.61        47

    accuracy                           0.78       154
   macro avg       0.74      0.72      0.73       154
weighted avg       0.77      0.78      0.77       154



In [30]:
model = RandomForestClassifier(n_estimators=300,criterion='entropy',max_features='sqrt',min_samples_leaf=10,random_state=100)
model.fit(X_train,y_train)
y_predict = model.predict(X_test)

In [31]:
from sklearn import metrics
cm = metrics.confusion_matrix(y_test,y_predict)
print(cm)
accuracy = metrics.accuracy_score(y_test,y_predict)
print(accuracy)
report = metrics.classification_report(y_test,y_predict)
print(report)

[[97 10]
 [17 30]]
0.8246753246753247
              precision    recall  f1-score   support

           0       0.85      0.91      0.88       107
           1       0.75      0.64      0.69        47

    accuracy                           0.82       154
   macro avg       0.80      0.77      0.78       154
weighted avg       0.82      0.82      0.82       154



Randomized Search CV

In [33]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV

In [51]:
# Number of trees in randomForest

n_estimaters = [int(x) for x in np.linspace(start=200,stop=2000,num=10)]
n_estimaters

# Number of features consider at every split
max_features = ['auto','sqrt','log2']

# Maximum numers of level in the tree
max_depth = [int(x) for x in np.linspace(10,1000,10)]

#Minimum numbers of samples required to split the node
min_samples_split = [1,2,3,4,6,7,8]

# Minimun nubers of samples required at each leaf node
min_samples_leaf =[1,2,4,6,8]

random_grid = {
    'n_estimators':n_estimaters,
    'max_features':max_features,
    'max_depth':max_depth,
    'min_samples_split':min_samples_split,
    'min_samples_leaf':min_samples_leaf,
    'criterion':['entropy','gini']}
print(random_grid)


{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [1, 2, 3, 4, 6, 7, 8], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [52]:
rf = RandomForestClassifier()
rf_randomCV = RandomizedSearchCV(rf,param_distributions=random_grid,n_iter=100,cv=3,verbose=2,random_state=100,n_jobs=-1)
rf_randomCV.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [1, 2, 3, 4, 6, 7,
                                                              8],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   

In [53]:
rf_randomCV.best_params_

{'n_estimators': 1200,
 'min_samples_split': 7,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 890,
 'criterion': 'gini'}

In [54]:
rf_randomCV.best_estimator_

RandomForestClassifier(max_depth=890, max_features='sqrt', min_samples_split=7,
                       n_estimators=1200)

In [55]:
best_random_grid = rf_randomCV.best_estimator_

In [57]:
from sklearn import metrics
y_predict = best_random_grid.predict(X_test)
cm = metrics.confusion_matrix(y_test,y_predict)
print(cm)
accuracy = metrics.accuracy_score(y_test,y_predict)
print(accuracy)
report = metrics.classification_report(y_test,y_predict)
print(report)

[[96 11]
 [14 33]]
0.8376623376623377
              precision    recall  f1-score   support

           0       0.87      0.90      0.88       107
           1       0.75      0.70      0.73        47

    accuracy                           0.84       154
   macro avg       0.81      0.80      0.81       154
weighted avg       0.84      0.84      0.84       154



In [58]:
rf_randomCV.best_params_

{'n_estimators': 1200,
 'min_samples_split': 7,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 890,
 'criterion': 'gini'}

In [59]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'criterion':[rf_randomCV.best_params_['criterion']],
    'max_depth':[rf_randomCV.best_params_['max_depth']],
    'min_samples_leaf':[rf_randomCV.best_params_['min_samples_leaf'],
                        rf_randomCV.best_params_['min_samples_leaf']+2,
                        rf_randomCV.best_params_['min_samples_leaf']+4],
     'min_samples_split':[rf_randomCV.best_params_['min_samples_split'],
                        rf_randomCV.best_params_['min_samples_split']+2,
                        rf_randomCV.best_params_['min_samples_split']+4],
    'n_estimators':[rf_randomCV.best_params_['n_estimators'],rf_randomCV.best_params_['n_estimators']+200,rf_randomCV.best_params_['n_estimators']+100],
}
print(param_grid)

{'criterion': ['gini'], 'max_depth': [890], 'min_samples_leaf': [1, 3, 5], 'min_samples_split': [7, 9, 11], 'n_estimators': [1200, 1400, 1300]}


In [64]:
rf = RandomForestClassifier()
rf_gridCV = GridSearchCV(rf,param_grid=param_grid,cv=3,verbose=2,n_jobs=-1)
rf_gridCV.fit(X_train,y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini'], 'max_depth': [890],
                         'min_samples_leaf': [1, 3, 5],
                         'min_samples_split': [7, 9, 11],
                         'n_estimators': [1200, 1400, 1300]},
             verbose=2)

In [65]:
rf_gridCV.best_params_

{'criterion': 'gini',
 'max_depth': 890,
 'min_samples_leaf': 3,
 'min_samples_split': 11,
 'n_estimators': 1400}

In [68]:
best_grid = rf_gridCV.best_estimator_

In [69]:
from sklearn import metrics
y_predict = best_grid.predict(X_test)
cm = metrics.confusion_matrix(y_test,y_predict)
print(cm)
accuracy = metrics.accuracy_score(y_test,y_predict)
print(accuracy)
report = metrics.classification_report(y_test,y_predict)
print(report)

[[96 11]
 [15 32]]
0.8311688311688312
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       107
           1       0.74      0.68      0.71        47

    accuracy                           0.83       154
   macro avg       0.80      0.79      0.80       154
weighted avg       0.83      0.83      0.83       154

